# SparkSession

U uvodnom tekstu smo ukratko spomenuli Spark ulaznu točku zvanu SparkContext koja predstavlja konekciju sa Spark klasterom - SparkSession je nadskup toga. Okružuje SparkContext i pruža mogućnost interakcije sa Spark SQL API-jem koji sadrži DF koji ćemo koristiti u većini naših programa. 

Prije pokretanja PySpark koda potrebno je kreirati SparkSession instancu. Dok je pokrenuta možete dohvatiti pregled vašeg Spark klastera i svih operacija na http://localhost:4040.

SparkSession objektu ćete dodijeliti određene parametre kako biste postavili i konfigurirali Spark aplikaciju - neki od parametara uključuju naziv aplikacije, URL i postavke alokacije memorije.Tako naša aplikacija postavlja sljedeće parametre:
* `builder` - objekt za kreiranje i konfiguraciju SparkSessiona
* `master("local[*]")` - lokalno izvršavanje aplikacije gdje * označava izrvšavanje aplikacije na svim dostupnim jezgrama - ovdje možete upisati i broj jezgri
* `appName("SparkVjezba")` - ime aplikacije
* `getOrCreate()` - kreira novi SparkSession ako ne postoji ili vraća postojeći ako se već izvodi te time osigurava da je samo jedan SparkSession objekt kreiran za aplikaciju što je važno za efikasnu upotrebu resursa. 

In [1]:
import findspark
from pyspark.sql import SparkSession

findspark.init()

spark = SparkSession.builder.master("local[*]").appName("SparkVjezba").getOrCreate()

# DataFrame

Ako ste korisitli Pandas biblioteku u Pythonu vjerojatno ste upoznati s konceptom DataFramea. Ipak, važno je naglasiti da DataFrame u Pandasu i Sparku nisu isti; neke od razlika su:
| Aspekt | Pandas DataFrame | Spark DataFrame |
|:--------:|:-----------------:|:----------------:|
| Izvršavanje | Manji setovi podataka i izvršavanje na jednom stroju | Veliki setovi podataka i izvršavanje na više storjeva |
| Evaluacija | Agilna | Lijena |
| Brzina | Brži za manje skupove podataka| Brži za veće skupove podataka |
| Izvori podataka | Zahtijeva dodatne bibioloteke za čitanje i pisanje raznih izvora podataka | Ugrađena potpora za čitanje i pisanje raznih izvora podataka| |

<br/>
Dataframe u Sparku možete shvatiti kao tablicu s redovima i stupcima koja sadrži veliku količinu podataka. Obrada podataka u Dataframeu se vrši korištenjem transformacija i akcija, a rezultat se može spremiti u novi Dataframe. Dataframe je također vrlo fleksibilan i može se koristiti za razne obrade podataka kao što su spajanje, filtriranje, sortiranje, agregacija i grupiranje podataka. 

## Kreiranje i korištenje DataFramea 

In [23]:
# kreacija DataFramea
data = [(1,2), (3,4)]
schema = "neparni int, parni int"
df_list = spark.createDataFrame(data, schema)

# prikaz DataFramea
df_list.show()

# prikaz scheme DataFramea
df_list.printSchema()

+-------+-----+
|neparni|parni|
+-------+-----+
|      1|    2|
|      3|    4|
+-------+-----+

root
 |-- neparni: integer (nullable = true)
 |-- parni: integer (nullable = true)



In [24]:
# selekcija stupaca
df_list.select(['neparni']).show()

+-------+
|neparni|
+-------+
|      1|
|      3|
+-------+



In [25]:
# dodavanje stupaca
from pyspark.sql.functions import col

df_list.withColumn("zbroj", col("neparni") + col("parni")).show()

+-------+-----+-----+
|neparni|parni|zbroj|
+-------+-----+-----+
|      1|    2|    3|
|      3|    4|    7|
+-------+-----+-----+



Prilikom stvaranja DF-a ukoliko nema zaglavlja (ili se ona ne čitaju) zadana vrijednost ukoliko se DF stvara iz CSV datoteke je _cn - gdje n predstavlja broj stupca počevši od nule, dok ukoliko se stvara iz liste zadana vrijednost je _n - gdje je n ekvivalent prethodnom n-u.

In [26]:
# preimenovanje stupaca
df_list_n = spark.createDataFrame(data)
df_list_n.withColumnRenamed('_1', 'neparni') \
         .withColumnRenamed('_2', 'parni') \
         .show()

+-------+-----+
|neparni|parni|
+-------+-----+
|      1|    2|
|      3|    4|
+-------+-----+



## Spark SQL i Window funkcije

Windowing u Sparku omogućuje obradu podataka u prozorima; odnosno, funkcije se mogu primijeniti i na više elemenata, a ne samo na pojedinačni element.

In [27]:
employee_data = (("John", "Engineering", 7000),
                 ("Jane", "Marketing", 5000),
                 ("Bob", "Engineering", 6000),
                 ("Mary", "Sales", 5500),
                 ("Alex", "Marketing", 4500),
                 ("Mike", "Engineering", 7500),
                 ("Julie", "Sales", 6000),
                 ("Sam", "Finance", 6500),
                 ("Lisa", "Marketing", 4500),
                 ("Tom", "Engineering", 8000))

columns = ["name", "department", "salary"]

df_employees = spark.createDataFrame(employee_data, columns)

### Funkcije rangiranja

Funkcija row_number koristi se za dodjelu slijednog broja retka koji počinje od 1 - svaki broj koristi samo jednom neovisno o tome je li došlo do izjenačenja.

In [28]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

window_order = Window.partitionBy("department").orderBy("salary")

df_employees.withColumn("row_number", row_number().over(window_order)).show()

+-----+-----------+------+----------+
| name| department|salary|row_number|
+-----+-----------+------+----------+
|  Bob|Engineering|  6000|         1|
| John|Engineering|  7000|         2|
| Mike|Engineering|  7500|         3|
|  Tom|Engineering|  8000|         4|
|  Sam|    Finance|  6500|         1|
| Alex|  Marketing|  4500|         1|
| Lisa|  Marketing|  4500|         2|
| Jane|  Marketing|  5000|         3|
| Mary|      Sales|  5500|         1|
|Julie|      Sales|  6000|         2|
+-----+-----------+------+----------+



Funkcije rank i dense_rank dodjeljuju rang rezultatu unutar prozora. Dok rank ostavlja prazna mjesta ako dolazi do izjednačenja, dense_rank preskače praznine.

In [29]:
from pyspark.sql.functions import rank, dense_rank

df_employees.withColumn("rank", rank().over(window_order)) \
            .withColumn("dense_rank", dense_rank().over(window_order)) \
            .show()

+-----+-----------+------+----+----------+
| name| department|salary|rank|dense_rank|
+-----+-----------+------+----+----------+
|  Bob|Engineering|  6000|   1|         1|
| John|Engineering|  7000|   2|         2|
| Mike|Engineering|  7500|   3|         3|
|  Tom|Engineering|  8000|   4|         4|
|  Sam|    Finance|  6500|   1|         1|
| Alex|  Marketing|  4500|   1|         1|
| Lisa|  Marketing|  4500|   1|         1|
| Jane|  Marketing|  5000|   3|         2|
| Mary|      Sales|  5500|   1|         1|
|Julie|      Sales|  6000|   2|         2|
+-----+-----------+------+----+----------+



Funkcija percent_rank namijenjena je za izračunavanje percentila za svaki redak unutar prozora.

In [30]:
from pyspark.sql.functions import percent_rank

df_employees.withColumn("percent_rank", percent_rank().over(window_order)).show()

+-----+-----------+------+------------------+
| name| department|salary|      percent_rank|
+-----+-----------+------+------------------+
|  Bob|Engineering|  6000|               0.0|
| John|Engineering|  7000|0.3333333333333333|
| Mike|Engineering|  7500|0.6666666666666666|
|  Tom|Engineering|  8000|               1.0|
|  Sam|    Finance|  6500|               0.0|
| Alex|  Marketing|  4500|               0.0|
| Lisa|  Marketing|  4500|               0.0|
| Jane|  Marketing|  5000|               1.0|
| Mary|      Sales|  5500|               0.0|
|Julie|      Sales|  6000|               1.0|
+-----+-----------+------+------------------+



Funkcija prozora ntile koristi se za podjelu rezultata unutar prozora u n jednakih dijelova.

In [31]:
from pyspark.sql.functions import ntile

df_employees.withColumn("ntile", ntile(2).over(window_order)).show()

+-----+-----------+------+-----+
| name| department|salary|ntile|
+-----+-----------+------+-----+
|  Bob|Engineering|  6000|    1|
| John|Engineering|  7000|    1|
| Mike|Engineering|  7500|    2|
|  Tom|Engineering|  8000|    2|
|  Sam|    Finance|  6500|    1|
| Alex|  Marketing|  4500|    1|
| Lisa|  Marketing|  4500|    1|
| Jane|  Marketing|  5000|    2|
| Mary|      Sales|  5500|    1|
|Julie|      Sales|  6000|    2|
+-----+-----------+------+-----+



### Analitičke funkcije 

Funkcija cume_dist koristi se za izračun kumulativne distribucije vrijednosti unutar svakog prozora.

In [32]:
from pyspark.sql.functions import cume_dist

df_employees.withColumn("cume_dist", cume_dist().over(window_order)).show()

+-----+-----------+------+------------------+
| name| department|salary|         cume_dist|
+-----+-----------+------+------------------+
|  Bob|Engineering|  6000|              0.25|
| John|Engineering|  7000|               0.5|
| Mike|Engineering|  7500|              0.75|
|  Tom|Engineering|  8000|               1.0|
|  Sam|    Finance|  6500|               1.0|
| Alex|  Marketing|  4500|0.6666666666666666|
| Lisa|  Marketing|  4500|0.6666666666666666|
| Jane|  Marketing|  5000|               1.0|
| Mary|      Sales|  5500|               0.5|
|Julie|      Sales|  6000|               1.0|
+-----+-----------+------+------------------+



Funkcija lag se koristi za dobivanje n-tog prethodnika reda unutar prozora, dok ćete funkciju lead iskoristiti kada želite dobit n-tog sljedbenika reda.

In [33]:
from pyspark.sql.functions import lag, lead

df_employees.withColumn("lag", lag("salary", 1).over(window_order)) \
            .withColumn("lead", lead("salary", 1).over(window_order)) \
            .show()

+-----+-----------+------+----+----+
| name| department|salary| lag|lead|
+-----+-----------+------+----+----+
|  Bob|Engineering|  6000|null|7000|
| John|Engineering|  7000|6000|7500|
| Mike|Engineering|  7500|7000|8000|
|  Tom|Engineering|  8000|7500|null|
|  Sam|    Finance|  6500|null|null|
| Alex|  Marketing|  4500|null|4500|
| Lisa|  Marketing|  4500|4500|5000|
| Jane|  Marketing|  5000|4500|null|
| Mary|      Sales|  5500|null|6000|
|Julie|      Sales|  6000|5500|null|
+-----+-----------+------+----+----+



### Agregatne funkcije



In [34]:
from pyspark.sql.functions import avg, sum, min, max

window_aggregation  = Window.partitionBy("department")

df_employees.withColumn("row", row_number().over(window_order)) \
            .withColumn("avg", avg(col("salary")).over(window_aggregation)) \
            .withColumn("sum", sum(col("salary")).over(window_aggregation)) \
            .withColumn("min", min(col("salary")).over(window_aggregation)) \
            .withColumn("max", max(col("salary")).over(window_aggregation)) \
            .where(col("row") == 1).select("department", "avg", "sum", "min", "max") \
            .show()

+-----------+-----------------+-----+----+----+
| department|              avg|  sum| min| max|
+-----------+-----------------+-----+----+----+
|Engineering|           7125.0|28500|6000|8000|
|    Finance|           6500.0| 6500|6500|6500|
|  Marketing|4666.666666666667|14000|4500|5000|
|      Sales|           5750.0|11500|5500|6000|
+-----------+-----------------+-----+----+----+



## Čitanje iz datoteke
PySpark posjeduje kapacitet otkrivanja scheme - možete uključiti tu opciju postavljajući inferSchema na True. Ovaj opcionalni parametar tjera PySpark da prođe kroz podatke dvaput - prvi put kako bi postavio tip svakog stupca i drugi put da bi učitao podatke. Time učitavanje traje duže, ali nam pomaže izbjeći ručno pisanje scheme. Preuzmite skup podataka sa sljedeće poveznice https://www.kaggle.com/datasets/crawford/80-cereals te ga stavite u mapu u kojoj se nalazi vaša bilježnica.

In [35]:
# Učitavanje datoteke sa zaglavljem i otkrivanjem scheme
df_cereal = spark.read.format('csv').option('header', True).option('inferSchema', True).load('cereal.csv')

# Prikaz samo prva tri reda
df_cereal.show(3)

# Primijetite tipove 
df_cereal.printSchema()

+-----------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|             name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+-----------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|        100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|         All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
+-----------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
only showing top 3 rows

root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- calories: integer 

## Agregacije nad grupiranim stupcem
Ako želite grupirati stupce možete se poslužiti transformacijom `groupby('imestupca')`. Metoda `agg()` (agg iz "aggregation") prihvaća jednu ili više agregatnih funkcija iz modula `pyspark.sql.functions`. U primjeru ispod računamo prosječni broj kalorija po tipu žitarica te novi stupac imenujemo metodom `alias('imestupca')` koji bi u protivnom nosio naziv prema funkcijama izvedenim u `agg()` metodi - u konkretnom slučaju "round(avg(calories), 2)".

In [36]:
from pyspark.sql.functions import round

df_cereal.groupBy('type') \
            .agg(round(avg('calories'), 2).alias('avg_calories')) \
            .show()

+----+------------+
|type|avg_calories|
+----+------------+
|   C|      107.16|
|   H|       100.0|
+----+------------+



## Važno! 
Prije nastavka rješavanja obavite zadatak iz poglavlja "*Povezivanje PySparka s Kafkom*". Obratite pozornost na stupce u datoteci - ako ste čitali cijelu vrijednost cijeli redak će vam biti zapisan pod jedan stupac. Razdvojite datoteku po stupcima prije nastavka - npr. spremanjem datoteke nanovo u CSV formatu.
## Dnevni prosjek 
Vaš zadatak je prikazati kretanje prosječne dnevne cijene kroz proteklih sto dana koristeći DataFrame kao strukturu podataka. Ispišite vaše rješenje na ekran - poredajte elemente silazno po prosječnoj cijeni zaokruženoj na jednu decimalu te im dodijelite rang. Možete koristiti `from_unixtime` iz `pyspark.sql.functions` modula za izvršenje zadatka.

In [37]:
# prostor za rješavanje

from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import round

ethDF = spark.read.option('header', False).csv("part-*.csv")

print(f"Dataframe rows: {ethDF.count()}")
ethDF = ethDF.withColumnRenamed('_c0', 'time').withColumnRenamed('_c1', 'priceUsd')

ethDF_result1 = ethDF.withColumn("date", from_unixtime(col("time") / 1000, "dd-MM-yyyy")) \
    .groupBy('date') \
    .agg(round(avg('priceUsd'), 1).alias('avg_priceUsd')) \
    .orderBy(col('avg_priceUsd').desc())

result1Window = Window.orderBy(col("avg_priceUsd").desc())
ethDF_result1 \
    .withColumn("rank", rank().over(result1Window)) \
    .show()


Dataframe rows: 2400
+----------+------------+----+
|      date|avg_priceUsd|rank|
+----------+------------+----+
|16-04-2023|      2101.5|   1|
|15-04-2023|      2100.1|   2|
|18-04-2023|      2093.0|   3|
|17-04-2023|      2093.0|   3|
|14-04-2023|      2092.8|   5|
|19-04-2023|      2033.8|   6|
|13-04-2023|      1963.1|   7|
|20-04-2023|      1953.3|   8|
|21-04-2023|      1917.5|   9|
|11-04-2023|      1916.2|  10|
|05-04-2023|      1905.9|  11|
|12-04-2023|      1889.4|  12|
|06-04-2023|      1886.5|  13|
|23-04-2023|      1873.6|  14|
|26-04-2023|      1873.2|  15|
|08-04-2023|      1870.9|  16|
|10-04-2023|      1868.6|  17|
|07-04-2023|      1865.3|  18|
|22-04-2023|      1863.5|  19|
|24-04-2023|      1855.2|  20|
+----------+------------+----+
only showing top 20 rows



# Resilient Distributed Dataset (RDD) 
Postoje dva načina kako kreirati RDD: 
* paralelizacijom postojeće kolekcije - pozivanjem metode parallelize
* referenciranjem skupa podataka u vanjsokom sustavu za pohranu - pozivanjem metode textfile koja kao argument prihvaća put do datoteke


RDD se poziva nad SparkContextom koji možete izvući iz SparkSession objekta.

In [77]:
# SparkContext objekt iz SparkSessiona
sc = spark.sparkContext

# Paralelizacija postojeće kolekcije
rdd_array = sc.parallelize([1, 2, 3])

# Učitavanje iz datoteke - zamijenite naziv CSV datoteke sa svojim
rdd_csv = sc.textFile("part-*.csv")

## Osnovne akcije nad RDD-ovima 
Metoda `collect()` dohvaća cijeli RDD (primijetite kako je svaki redak pročitan kao jedan element iako sadrži dva podatka). 

In [61]:
rdd_csv.collect()

['1673827200000,1552.7180944035117',
 '1673830800000,1555.8932523581448',
 '1673856000000,1566.6530104326284',
 '1673874000000,1542.0712593148153',
 '1673888400000,1551.1278704476324',
 '1673906400000,1579.841974548664',
 '1673913600000,1576.8727145532227',
 '1673931600000,1567.456980698864',
 '1673946000000,1569.2417714222902',
 '1673949600000,1565.7563629363717',
 '1673971200000,1572.2463728025753',
 '1673974800000,1575.336861517224',
 '1673978400000,1578.205179564794',
 '1673985600000,1588.548326587762',
 '1674025200000,1581.449374348677',
 '1674039600000,1575.5110607481354',
 '1674093600000,1518.3002957091646',
 '1674100800000,1525.5019117961906',
 '1674111600000,1531.2619447847678',
 '1674118800000,1528.2121680103821',
 '1674126000000,1525.69969560511',
 '1674190800000,1549.2764094266067',
 '1674205200000,1552.0818532472658',
 '1674223200000,1555.6578417077346',
 '1674226800000,1561.332016701784',
 '1674234000000,1587.6743941711006',
 '1674259200000,1660.8224710051848',
 '16742628

Metoda `count()` pokazuje broj elemenata u RDD-u.

In [62]:
rdd_csv.count()

2400

Kako biste uzeli određeni broj elemenata (po redu) potrebno je pozvati `take(n)` gdje n ukazuje koji je to broj elemenata:

In [63]:
rdd_csv.take(2)

['1673827200000,1552.7180944035117', '1673830800000,1555.8932523581448']

## Transformacije nad RDD-ovima

Tranformacija filter se koristi kako bi se zadržali elementi koji ispunjavaju određeni uvjet; nad svakim elementom se poziva funkcija te ako je odgovor na tu funkciju True taj element se zadržava.

In [106]:
veci_od_jedan = rdd_array.filter(lambda x: x > 1)
veci_od_jedan.collect()

[2, 3]

Koristeći map transformaciju možete nad svakim elementom RDD-a izvesti određenu funkciju. Primjerice, ako imate csv datoteku u kojoj imate više stupaca - te elemente možete pročitati pozivajući funkciju koja će razdvojiti vaš redak u elemente po delimiteru u vašoj datoteci.

Kao vježbu napišite naredbu s lambda funkcijom koja će vaš rdd_csv razdvojiti po stupcima te prikažite novi RDD na zaslonu.

In [78]:
# Prostor za rješavanje

rdd_csv = rdd_csv.map(lambda f : f.split(","))

rdd_csv.collect()

[['1673827200000', '1552.7180944035117'],
 ['1673830800000', '1555.8932523581448'],
 ['1673856000000', '1566.6530104326284'],
 ['1673874000000', '1542.0712593148153'],
 ['1673888400000', '1551.1278704476324'],
 ['1673906400000', '1579.841974548664'],
 ['1673913600000', '1576.8727145532227'],
 ['1673931600000', '1567.456980698864'],
 ['1673946000000', '1569.2417714222902'],
 ['1673949600000', '1565.7563629363717'],
 ['1673971200000', '1572.2463728025753'],
 ['1673974800000', '1575.336861517224'],
 ['1673978400000', '1578.205179564794'],
 ['1673985600000', '1588.548326587762'],
 ['1674025200000', '1581.449374348677'],
 ['1674039600000', '1575.5110607481354'],
 ['1674093600000', '1518.3002957091646'],
 ['1674100800000', '1525.5019117961906'],
 ['1674111600000', '1531.2619447847678'],
 ['1674118800000', '1528.2121680103821'],
 ['1674126000000', '1525.69969560511'],
 ['1674190800000', '1549.2764094266067'],
 ['1674205200000', '1552.0818532472658'],
 ['1674223200000', '1555.6578417077346'],


Transformacija reduce primjenjuje danu funkciju na dva parametra istog tipa te vraća jedan rezultat. Ta funkcija se primjenjuje elementima u strukturi sličnoj stablu, gdje se na svakoj razini broj elemenata prepolavlja sve dok se ne dođe do jednog elementa.

In [66]:
sum = rdd_array.reduce(lambda x, y: x + y)
print(sum)

6


Transformacija groupByKey grupira zapise prema ključu i vraća RDD gdje svaki zapis ima ključ i listu vrijednosti povezanih s tim ključem.

In [67]:
rdd_employees = sc.parallelize([("Engineering", "Mike"), ("Engineering", "Tom"), ("Marketing", "Lisa")])

for key, values in rdd_employees.groupByKey().collect():
    print(f"{key}: {list(values)}")

Engineering: ['Mike', 'Tom']
Marketing: ['Lisa']


## Najviše dnevne cijene

Vaš zadatak je pronaći najveći dnevni maksimum kroz protekla dva tjedna koristeći RDD kao strukturu podataka. Pripazite na tipove učitanih podataka. Ovisno o kodu koji unesete možete nepovratno "pokvariti" kernel kada računate s RDD-om u Jupyteru pa ga stoga iznova pokrenite s vremena na vrijeme.

In [112]:
# prostor za rješavanje

import datetime
import builtins

twoWeeksBeforeDate = datetime.date.today() - datetime.timedelta(days=14)
print(twoWeeksBeforeDate)

rdd_result = rdd_csv.map(lambda a : [int(int(a[0]) / 1000), float(a[1])]) \
    .map(lambda a : [a[0], a[1], datetime.date.fromtimestamp(a[0])]) \
    .filter(lambda a : ((a[2] - twoWeeksBeforeDate).days >= 0) & ((a[2] - twoWeeksBeforeDate).days < 14)) \
    .map(lambda a : [a[1], a[2], a[0]])
rdd_result.collect()

max_priceUsd = rdd_result.reduce(builtins.max)
print(max_priceUsd)

2023-04-12
[2129.8594786972853, datetime.date(2023, 4, 17), 1681682400]
